---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    models = []
    for graph in P1_Graphs:
        degrees = graph.degree()
        unique_degrees = len(set(degrees.values()))
        clustering = nx.average_clustering(graph)
        path = nx.average_shortest_path_length(graph)
        # PA model specifies graphs with power law distributions, which have heavy tails.
        # exploring the distribution values and coefficients showed that anything larger than 8 should be considered
        # a power law
        if unique_degrees>11:
            models.append('PA')
        # small world with high probability will show higher clustering
        elif clustering > 0.4:
            models.append('SW_L') 
        else:
            models.append('SW_H') 
    
    return models

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

def create_df_with_features():
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['betweeness_centrality'] = pd.Series(nx.betweenness_centrality(G))
    df['pr'] = pd.Series(nx.pagerank(G, alpha=0.8))
    df['hub'] = pd.Series(nx.hits(G)[0])
    return df
    
    
def salary_predictions():
    df = create_df_with_features()
    df_train = df[~pd.isnull(df['ManagementSalary'])]
    df_test = df[pd.isnull(df['ManagementSalary'])]
    
    
    clf_rf = RandomForestClassifier(n_estimators=500, random_state=42)
    clf_gbm = GradientBoostingClassifier()
    clf_rf.fit(df_train.drop('ManagementSalary', 1), df_train.ManagementSalary)
    clf_gbm.fit(df_train.drop('ManagementSalary', 1), df_train.ManagementSalary)
    rf_proba = clf_rf.predict_proba(df_test.drop('ManagementSalary', 1))[:, 1]
    gbm_proba = clf_gbm.predict_proba(df_test.drop('ManagementSalary', 1))[:, 1]
    
    return pd.Series((rf_proba + gbm_proba)/2, df_test.index)

In [9]:
salary_predictions()

1       0.014547
2       0.989867
5       0.994751
8       0.079328
14      0.039175
18      0.036173
27      0.041131
30      0.791729
31      0.383118
34      0.063405
37      0.033407
40      0.036186
45      0.013617
54      0.445882
55      0.658534
60      0.220007
62      0.994751
65      0.989741
77      0.030603
79      0.029599
97      0.006718
101     0.001465
103     0.484671
108     0.031527
113     0.045203
122     0.003939
141     0.267311
142     0.993751
144     0.017514
145     0.764718
          ...   
913     0.004411
914     0.003245
915     0.000866
918     0.066358
923     0.006328
926     0.028711
931     0.006645
934     0.000618
939     0.000620
944     0.000580
945     0.005328
947     0.258781
950     0.033586
951     0.002215
953     0.002063
959     0.000581
962     0.000647
963     0.221748
968     0.006954
969     0.010612
974     0.011516
984     0.002580
987     0.016453
989     0.007785
991     0.008121
992     0.000620
994     0.000496
996     0.0005

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [10]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [28]:
df=future_connections

In [33]:
def new_connections_predictions():
    # generate all edge connections
    df['pref_attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G, df.index))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G, df.index))]
    df['common_neighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    df['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G,
                                                                              df.index,
                                                                              community='Department')]
    
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    
    clf_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf_gbm = GradientBoostingClassifier()
    clf_rf.fit(df_train.drop('Future Connection', 1), df_train[['Future Connection']])
    clf_gbm.fit(df_train.drop('Future Connection', 1), df_train[['Future Connection']])
    rf_proba = clf_rf.predict_proba(df_test.drop('Future Connection', 1))[:, 1]
    gbm_proba = clf_gbm.predict_proba(df_test.drop('Future Connection', 1))[:, 1]
    
    return pd.Series((rf_proba + gbm_proba)/2, df_test.index)

In [34]:
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(107, 348)    0.014180
(542, 751)    0.013125
(20, 426)     0.490056
(50, 989)     0.013690
(942, 986)    0.012007
(324, 857)    0.014655
(13, 710)     0.095954
(19, 271)     0.147372
(319, 878)    0.012965
(659, 707)    0.013489
(49, 843)     0.013697
(208, 893)    0.009274
(377, 469)    0.007227
(405, 999)    0.012789
(129, 740)    0.009172
(292, 618)    0.023665
(239, 689)    0.009570
(359, 373)    0.090745
(53, 523)     0.187647
(276, 984)    0.010387
(202, 997)    0.011524
(604, 619)    0.059457
(270, 911)    0.013539
(261, 481)    0.048791
(200, 450)    0.982248
(213, 634)    0.015202
(644, 735)    0.064921
(346, 553)    0.006539
(521, 738)    0.015938
(422, 953)    0.007702
                ...   
(672, 848)    0.013539
(28, 127)     0.988429
(202, 661)    0.011775
(54, 195)     0.998959
(295, 864)    0.013301
(814, 936)    0.011979
(839, 874)    0.012007
(139, 843)    0.013483
(461, 544)    0.012728
(68, 487)     0.006913
(622, 932)    0.015036
(504, 936)    0.007933
(479, 528) 